In [73]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows',None)
# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

# Question 1

In [74]:
Energy = pd.read_excel('assets/Energy Indicators.xls', header = None, usecols='C:F', skiprows = 18, skipfooter = 38)
cols = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
Energy.columns = cols
Energy.set_index('Country', inplace=True)
ren  = {"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"}


In [75]:
def conversion(item):
    if item == '...':
        return None
    else:
        new_val =  item*1000000
        return new_val

In [83]:
Energy['Energy Supply'] = Energy['Energy Supply'].apply(conversion)

Energy Supply               2.554e+15
Energy Supply per Capita           28
% Renewable                   45.3215
Name: Viet Nam, dtype: object

In [77]:
def cleaning(item):

    result = re.sub('\d', '', item) #remove digits
    result = re.sub(r'\([^)]*\)', '', result) #Remove things in parenthesis
    result = result.rstrip()
    
    return result

Energy.index = Energy.index.map(cleaning)
Energy.rename(index = ren, inplace = True)


In [78]:
GDP = pd.read_csv('assets/world_bank.csv', skiprows=4, index_col= 0)

In [79]:
rena = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
dropcol =  list(range(2006,2016))
dropcol = map(str, dropcol)  
#GDP.drop(GDP.columns.difference(dropcol), 1, inplace= True)
GDP = GDP[dropcol]

In [80]:
GDP.rename(index = rena, inplace = True)
print(GDP.shape)
GDP.head()


(264, 10)


,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country Name,,,,,,,,,,
Aruba,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
Andorra,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
Afghanistan,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
Angola,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
Albania,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


In [81]:
ScimEn = pd.read_csv('assets/scimagojr-3.csv', index_col= 0)
print(ScimEn.shape)
ScimEn = ScimEn[ScimEn.index <= 15]
ScimEn = ScimEn.merge(GDP, how = 'inner', left_on = 'Country', right_on = 'Country Name')
ScimEn.set_index('Country')

ScimEn = ScimEn.merge(Energy, how = 'inner', left_on='Country', right_on = 'Country')


(191, 7)


In [82]:
ScimEn

,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Energy Supply,Energy Supply per Capita,% Renewable
0,China,127050,126767,597237,411683,4.70,138,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,1.271910e+11,93,19.754910
1,United States,96661,94747,792274,265436,8.20,230,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,9.083800e+10,286,11.570980
2,Japan,30504,30287,223024,61554,7.31,134,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,1.898400e+10,149,10.232820
3,United Kingdom,20944,20357,206091,37874,9.84,139,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,7.920000e+09,124,10.600470
4,Russian Federation,18534,18301,34266,12422,1.85,57,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,3.070900e+10,214,17.288680
5,Canada,17899,17620,215003,40930,12.01,149,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,1.043100e+10,296,61.945430
6,Germany,17027,16831,140566,27426,8.26,126,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,1.326100e+10,165,17.901530
7,India,15005,14841,128763,37209,8.58,115,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,3.319500e+10,26,14.969080
8,France,13153,12973,130632,28601,9.93,114,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,1.059700e+10,166,17.020280
9,South Korea,11983,11923,114675,22595,9.57,104,9.410199e+11,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12,1.100700e+10,221,2.279353
